# Join Tables from Two Data Sources

This notebook fetches data tables from 2 different sources and join them on a key column.

### Provide your watsonx.data credential

```
wx_data_host = "<FQDN>"
wx_data_port = "<PORT>"
wx_data_catalog = "<CATALOG_NAME>"
wx_data_schema = "<SCHEMA_NME>"
wx_data_user = "<USER_NAME>"
wx_data_password = "<USER_PASSWORD>"
```

In [1]:
# The code was removed by Watson Studio for sharing.

### Install SQLAlchemy and PyHive

In [2]:
# %pip install presto-python-client
%pip install sqlalchemy "pyhive[presto]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyhive: filename=PyHive-0.7.0-py3-none-any.whl size=53878 sha256=201372102431f49e36cb3c186afdc9d47c5bd157349216a135ecba0da7beb97b
  Stored in directory: /home/spark/shared/.cache/pip/wheels/d3/fc/31/6974270c69ccc5bf8f848e2e41b527d0e8f5b9b973696a29a9
Successfully built pyhive
Note: you may need to restart the kernel to use updated packages.


### Fetch data from SAP Datasphere through a Generic JDBC connector

In [3]:
import itc_utils.flight_service as itcfs

readClient = itcfs.get_flight_client()

# NOTE:
#  A limit of 90000 rows has been applied to the request to enable sample previewing.
#  Adjust the display message as needed by editing the following lines:
from IPython.display import display, HTML
display(HTML("A row limit of 90000 has been applied to the query to enable sample previewing. If the data set is larger, only the first 90000 rows will be loaded."))
#  Edit select_statement to change or disable the row limit.
#
SAP_Datasphere___Demo_data_request = {
    'connected_data_name': """epp_primary""",
    'interaction_properties': {
        'row_limit': 90000
    }
}

flightInfo = itcfs.get_flight_info(readClient, nb_data_request=SAP_Datasphere___Demo_data_request)

data_df_1 = itcfs.read_pandas_and_concat(readClient, flightInfo, timeout=240)
#data_df_1.columns = data_df_1.columns.str.upper()

data_df_1.head(5)

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,any_awards_won,is_promoted
0,45709,Sales & Marketing,region_31,Bachelor's,f,other,1,29,NaN,1,0,0
1,66874,Sales & Marketing,region_27,Bachelor's,f,other,1,30,NaN,1,0,0
2,36904,Sales & Marketing,region_15,Bachelor's,m,other,1,29,3.0,2,0,0
3,32877,Sales & Marketing,region_2,Bachelor's,f,other,1,40,3.0,12,0,0
4,58415,Sales & Marketing,region_7,Bachelor's,m,other,1,45,4.0,5,0,0


### Fetch data from Db2 through watsonx.data

In [4]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

In [5]:
connect_args = {
    'protocol': 'https'
    }

engine = create_engine(
    f"presto://{wx_data_user}:{wx_data_password}@{wx_data_host}:{wx_data_port}/{wx_data_catalog}/{wx_data_schema}",
    connect_args=connect_args
    )

In [6]:
data_df_2 = pd.read_sql_query('SELECT * from epp_extra', engine)
#data_df_2.columns = data_df_2.columns.str.upper()

data_df_2.head(5)

,employee_id,kpis_met_above_80_percent,avg_training_score
0,65438,1,49
1,65141,0,60
2,7513,0,50
3,2542,0,50
4,48945,0,73


### Join the data columns in Pandas

In [7]:
import pandas as pd

merged_data = pd.merge(data_df_1, data_df_2, on=['employee_id'])
merged_data.head(5)

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,any_awards_won,is_promoted,kpis_met_above_80_percent,avg_training_score
0,45709,Sales & Marketing,region_31,Bachelor's,f,other,1,29,NaN,1,0,0,0,49
1,66874,Sales & Marketing,region_27,Bachelor's,f,other,1,30,NaN,1,0,0,0,50
2,36904,Sales & Marketing,region_15,Bachelor's,m,other,1,29,3.0,2,0,0,0,51
3,32877,Sales & Marketing,region_2,Bachelor's,f,other,1,40,3.0,12,0,0,0,50
4,58415,Sales & Marketing,region_7,Bachelor's,m,other,1,45,4.0,5,0,0,0,50


In [8]:
merged_data.shape

(54808, 14)

In [9]:
merged_data.columns

Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'any_awards_won', 'is_promoted',
       'kpis_met_above_80_percent', 'avg_training_score'],
      dtype='object')

## Save the data as CSV file

In [10]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [11]:
from io import BytesIO

# write the dataframe to a buffer
buffer = BytesIO()
merged_data.to_csv(buffer)

# reset for subsequent reading
buffer.seek(0)

# Save the data to project
assetname="merged_data.csv"
wslib.save_data(assetname, data=buffer.read(), overwrite=True)

{'name': 'merged_data.csv',
 'asset_type': 'data_asset',
 'asset_id': 'e0449e6f-3f93-414f-9eaf-1fae7d6262f4',
 'attachment_id': '3e59656f-9338-400e-af6e-b7ef07a36ad5',
 'filepath': 'merged_data.csv',
 'data_size': None,
 'mime': 'text/csv',
 'summary': ['created or overwritten file',
  'created data asset',
  'created attachment']}